In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def backgroundFinder(video, upperlim, retframe):
    """
    video: relative pathway to the video to load in.
    upperlim: when to stop running the video, default = 1000 frames.
    while recursiveBStrack should be able to handle slight
    movements to the arena, it is still best to choose a background that
    the majority of the video will live in.
    """
    video_capture = cv2.VideoCapture(video)
    
    frame_counter = 0

    # Loop to read and display each frame
    while frame_counter < upperlim:
        ret, frame = video_capture.read()
        if not ret:
            break  # Break the loop when no frames left

        # Display the frame
        cv2.imshow('Video', frame)

        # Increment frame counter
        frame_counter += 1

        # Display frame counter
        cv2.putText(frame, f'Frame: {frame_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Video', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    video_capture.release()
    cv2.destroyAllWindows()

    video_capture = cv2.VideoCapture(video)


    frames = []

    for i in range(retframe):
        frames.append(video_capture.read()[1])

    video_capture.release()

    background = np.array(frames[-1])

    return background


def RecursiveeBSTracker(video, initbackground, centroid_size, crop = None, start = 0):
    """    
    video: a relative pathway to the video to process
    initbackground: the cv2 image of the background, obtained 
    using the return of the backgroundFinder function
    crop: a list storing the pixel index for the top, bottom, left, 
    and right bounds of the arena (all else will be cropped)
    centroid_size: determines the size of the box which will encapsulate
    the center mass of brightness in the current frame. background will
    be updated with everything outside of this box. ** this parameter
    should scale with the pixel-size of your animal! **
    """
    video_capture = cv2.VideoCapture(video)

    backcropped = initbackground

    if crop:
        backcropped = initbackground[crop[0]:crop[1],crop[2]:crop[3]]

    subtract_video = []
    signed_background = backcropped.astype(np.int16)

    frameCounter = 0

    while True:
        ret, frame = video_capture.read()
        
        if not ret:
            break  # Break the loop when no frames left
        
        if frameCounter < start:
            continue

        if crop:
    
            frame = frame[crop[0]:crop[1],crop[2]:crop[3]]  # 330, 250

        #Condition for when to update background

        # Convert each element in the array to an integer
        if (frameCounter%5==0):
            subtract_video.append(np.clip(np.clip(np.abs(frame.astype(np.int16) - signed_background)-20, 0, None)*2, 0, 255).astype(np.uint8))
        
        if(frameCounter%200==0 and frameCounter>1000):
            gray_subtract_last = cv2.cvtColor(subtract_video[-1], cv2.COLOR_BGR2GRAY)

            # this centroid pair will onl;y be used for updating background
            moments = cv2.moments(gray_subtract_last)
            if moments['m00'] != 0:
                centroid_x = int(moments['m10'] / moments['m00'])
                centroid_y = int(moments['m01'] / moments['m00'])
            else:
                centroid_x, centroid_y = None, None  # Handle the case of no bright regions

            leftBound = np.clip(centroid_x - centroid_size, 0, None)
            rightBound = np.clip(centroid_x + centroid_size, None, 249)
            bottomBound = np.clip(centroid_y + centroid_size, None, 329)
            topBound = np.clip(centroid_y - centroid_size, 0, None)


            #cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw a red circle

            #cv2.rectangle(frame, (leftBound,topBound), (rightBound,bottomBound), (0, 255, 0), 3)  # Green square with thickness of 3


            # take everything in grayscale_frame that is outside of left, right, top, bottom bounds and update signed_background

            for x in range(0,250):
                for y in range(0,330):
                    # if we are outside
                    if not ((x>leftBound) and (x<rightBound) and (y<bottomBound) and (y>topBound) ):

                        signed_background[y][x] = frame[y][x]

        
        frameCounter = frameCounter+1

    return subtract_video

def watch(processedVid, tracker, trackerSize, trackerColor = (0,0,255)):
    """
    processedVid: output of RecursiveBSTracker
    tracker: bool, do you want a center of mass tracker? 
    (you should select no if using this for preprocessing e.g.,
    for use in SLEAP or some similar deep learning program)
    trackerColor: RGB value for tracker
    trackerSize: marker size for tracker circle
    """

    subtract_vid = processedVid

    if tracker:
        gray_subtract = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in processedVid] 

        for i in range(len(gray_subtract)):

            moments = cv2.moments(gray_subtract[i])
            if moments['m00'] != 0:
                centroid_x = int(moments['m10'] / moments['m00'])
                centroid_y = int(moments['m01'] / moments['m00'])
            else:
                centroid_x, centroid_y = None, None  # Handle the case of no bright regions

            cv2.circle(subtract_vid[i], (centroid_x, centroid_y), trackerSize, trackerColor, -1)  # Draw a red circle



    for frame in subtract_vid:
        cv2.imshow('Modified Frame', frame) # 330 x 250
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
    


In [3]:
# Load the video
video_capture = cv2.VideoCapture('./2_2023-07-31_18-14-52_F2_BL1_EdgeGravel.avi')

In [4]:
relativeCray = './2_2023-07-31_18-14-52_F2_BL1_EdgeGravel.avi'
relCrayCrop = [120,450,170,420]
relCrayCentrSz = 60
initbackground = backgroundFinder(relativeCray,10, 250)

In [5]:
processedVideo = RecursiveeBSTracker(relativeCray, initbackground, relCrayCentrSz, relCrayCrop)

In [15]:
watch(processedVideo, True, 2, (190,170,50))

KeyboardInterrupt: 

# RAT

In [24]:
relativeRat = 'R45_Male_Water_Round_3_Camera_3_Separate_2023-12-18.avi'
relRatCrop = None
relRatCentrSz = 60
initbackground = backgroundFinder(relativeRat,10, 1000)

In [25]:
processedVideo = RecursiveeBSTracker(relativeRat, initbackground, relRatCentrSz, start = 1000)

In [ ]:
watch(processedVideo, True, 2, (190,170,50))

In [3]:
frames = []

for i in range(250):
    frames.append(video_capture.read()[1])

video_capture.release()

background = np.array(frames[-1])

#print(background)


In [32]:
[print(background[10:20][i][10:20]) for i in range(10)]
pass

[[ 75  78  76]
 [ 76  79  77]
 [ 71  74  72]
 [ 72  75  73]
 [ 73  76  74]
 [ 64  67  65]
 [192 186 181]
 [255 250 245]
 [245 254 248]
 [251 255 254]]
[[73 76 74]
 [78 81 79]
 [76 79 77]
 [72 75 73]
 [70 73 71]
 [70 73 71]
 [77 71 66]
 [73 67 62]
 [53 62 56]
 [57 66 60]]
[[94 97 95]
 [75 78 76]
 [71 74 72]
 [58 61 59]
 [72 75 73]
 [59 62 60]
 [64 71 59]
 [55 62 50]
 [69 63 66]
 [73 67 70]]
[[84 87 85]
 [78 81 79]
 [82 85 83]
 [85 88 86]
 [72 75 73]
 [69 72 70]
 [69 76 64]
 [68 75 63]
 [78 72 75]
 [71 65 68]]
[[86 89 87]
 [88 91 89]
 [82 85 83]
 [84 87 85]
 [69 72 70]
 [66 69 67]
 [85 77 67]
 [83 75 65]
 [54 67 75]
 [49 62 70]]
[[91 94 92]
 [79 82 80]
 [86 89 87]
 [73 76 74]
 [69 72 70]
 [58 61 59]
 [74 66 56]
 [83 75 65]
 [57 70 78]
 [49 62 70]]
[[86 89 87]
 [83 86 84]
 [78 81 79]
 [75 78 76]
 [71 74 72]
 [70 73 71]
 [66 69 67]
 [62 65 63]
 [59 62 60]
 [57 60 58]]
[[85 88 86]
 [80 83 81]
 [77 80 78]
 [72 75 73]
 [70 73 71]
 [68 71 69]
 [65 68 66]
 [62 65 63]
 [59 62 60]
 [57 60 58]]
[[

In [34]:
frames = []

for i in range(1000):
    frames.append(video_capture.read()[1])

video_capture.release()

frame1000 = np.array(frames[-1])

#print(background)


In [38]:
for i in range(1):
    print(frame1000[10:20][i][10:20])

    print(background[10:20][i][10:20])
    
    print(np.abs(np.array(frame1000[10:20][i][10:20]) - np.array(background[10:20][i][10:20])))
    


[[ 73  77  72]
 [ 64  68  63]
 [ 73  77  72]
 [ 76  80  75]
 [ 79  83  78]
 [ 62  66  61]
 [190 186 182]
 [255 251 247]
 [251 253 246]
 [245 247 240]]
[[ 75  78  76]
 [ 76  79  77]
 [ 71  74  72]
 [ 72  75  73]
 [ 73  76  74]
 [ 64  67  65]
 [192 186 181]
 [255 250 245]
 [245 254 248]
 [251 255 254]]
[[254 255 252]
 [244 245 242]
 [  2   3   0]
 [  4   5   2]
 [  6   7   4]
 [254 255 252]
 [254   0   1]
 [  0   1   2]
 [  6 255 254]
 [250 248 242]]


In [10]:
backcropped = background[120:450,170:420]

cv2.imshow('Modified Frame', backcropped) # 330 x 250
if cv2.waitKey(2225) & 0xFF == ord('q'):
    cv2.destroyAllWindows()

NameError: name 'backcropped' is not defined

In [19]:
# Load the video
video_capture = cv2.VideoCapture('./2_2023-07-31_18-14-52_F2_BL1_EdgeGravel.avi')

Background Differences

In [49]:
# Load the video
video_capture = cv2.VideoCapture('./2_2023-07-31_18-14-52_F2_BL1_EdgeGravel.avi')


In [6]:
print(type(background[0][0][0]))

<class 'numpy.uint8'>


In [50]:
subtract_video = []
signed_background = background.astype(np.int16)[120:450,170:420]

frameCounter = 0

while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break  # Break the loop when no frames left

    frame = frame[120:450,170:420]  # 330, 250

    #Condition for when to update background

    # Convert each element in the array to an integer
    if (frameCounter%5==0):
        subtract_video.append(np.clip(np.clip(np.abs(frame.astype(np.int16) - signed_background)-20, 0, None)*2, 0, 255).astype(np.uint8))
    
    if(frameCounter%200==0 and frameCounter>1000):
        gray_subtract_last = cv2.cvtColor(subtract_video[-1], cv2.COLOR_BGR2GRAY)

        # this centroid pair will onl;y be used for updating background
        moments = cv2.moments(gray_subtract_last)
        if moments['m00'] != 0:
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
        else:
            centroid_x, centroid_y = None, None  # Handle the case of no bright regions

        leftBound = np.clip(centroid_x - 60, 0, None)
        rightBound = np.clip(centroid_x + 60, None, 249)
        bottomBound = np.clip(centroid_y + 60, None, 329)
        topBound = np.clip(centroid_y - 60, 0, None)


        #cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw a red circle

        #cv2.rectangle(frame, (leftBound,topBound), (rightBound,bottomBound), (0, 255, 0), 3)  # Green square with thickness of 3


        # take everything in grayscale_frame that is outside of left, right, top, bottom bounds and update signed_background

        for x in range(0,250):
            for y in range(0,330):
                # if we are outside
                if not ((x>leftBound) and (x<rightBound) and (y<bottomBound) and (y>topBound) ):

                    signed_background[y][x] = frame[y][x]

    
    frameCounter = frameCounter+1





In [35]:
for frame in subtract_video[-1000:]:
    cv2.imshow('Modified Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Close OpenCV windows
cv2.destroyAllWindows()

KeyboardInterrupt: 

Weighed Sum

In [51]:
subtract_vid2 = subtract_video
# made grayscale list of frames for computing moment (needs grayscale)
gray_subtract = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in subtract_video] 

for i in range(len(gray_subtract)):

    moments = cv2.moments(gray_subtract[i])
    if moments['m00'] != 0:
        centroid_x = int(moments['m10'] / moments['m00'])
        centroid_y = int(moments['m01'] / moments['m00'])
    else:
        centroid_x, centroid_y = None, None  # Handle the case of no bright regions

    cv2.circle(subtract_vid2[i], (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw a red circle



In [59]:
# video is subtract_video

for frame in subtract_vid2[::5]:

    cv2.imshow('Modified Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Subsequent Differences

In [4]:
# Load the video
video_capture = cv2.VideoCapture('./2_2023-07-31_18-14-52_F2_BL1_EdgeGravel.avi')

In [5]:
subseq_video = []
ret, subframe = video_capture.read()

signed_background = background.astype(np.int16)

subframe_proper = subframe.astype(np.int16)

while True:
    
    ret, frame = video_capture.read()

    if not ret:
        break  # Break the loop when no frames left

    intframe = frame.astype(np.int16)

    diff = np.abs(intframe - subframe_proper).astype(np.uint8)
    subframe_proper = intframe

In [6]:
for frame in subseq_video:
    cv2.imshow('Modified Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Close OpenCV windows
cv2.destroyAllWindows()

In [3]:
frame_counter = 0

# Loop to read and display each frame
while frame_counter < 5000:
    ret, frame = video_capture.read()
    if not ret:
        break  # Break the loop when no frames left

    # Display the frame
    cv2.imshow('Video', frame)

    # Increment frame counter
    frame_counter += 1

    # Display frame counter
    cv2.putText(frame, f'Frame: {frame_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
# Create interactive figure to select background frame
background_image = select_background_frame(video_capture)

In [ ]:





# Reset video capture to beginning
video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Read the first frame to determine dimensions
ret, frame = video_capture.read()
height, width, _ = frame.shape

# Create a VideoWriter object to save the processed video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('processed_video.avi', fourcc, 30.0, (width, height))

# Process each frame
while True:
    ret, frame = video_capture.read()
    if not ret:
        break  # Break the loop when no frames left

    # Subtract background image from the frame
    processed_frame = cv2.absdiff(frame, background_image)

    # Write the processed frame to the output video
    output_video.write(processed_frame)

    # Display the processed frame
    cv2.imshow('Processed Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Break the loop if 'q' is pressed

# Release video capture and writer objects
video_capture.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
